In [1]:
import os

os.chdir("..")
os.chdir("..")
os.chdir("..")

In [ ]:
import os
import tkinter as tk
from tkinter import messagebox
from pathlib import Path
import pandas as pd
from PIL import Image, ImageTk
import json
import pygame
import random
from datetime import datetime
import time
from deep_translator import GoogleTranslator
from random import randint

from typing import List, Dict, Union

import csv
from typing import Dict, Union, List


# Inicializa o mixer do pygame
pygame.mixer.init()

TITLE_FONT = ("Arial", 20)
LABEL_FONT = ("Arial", 14)
SMALL_FONT = ("Arial", 12)

# DATA_PATH = Path("extract_data_video/data/extracted_data/words/data_organize")
DATA_PATH = Path("database/vocabulary/words/data_organize")
SUBCATEGORY_NAME = "compras_online"

def save_game_data(data: Dict[str, Union[str, int, float, List[str], bool]], 
                   file_path: str = "database/infos/game_data_{game_name}.csv") -> None:
    
    file_path = file_path.format(game_name=data.get("game_name", "error"))
    file_exists = os.path.isfile(file_path)

    # Convert listas para string
    for key, value in data.items():
        if isinstance(value, list):
            data[key] = ",".join(map(str, value))

    # Define os fieldnames dinamicamente
    if file_exists:
        with open(file_path, "r", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            fieldnames = reader.fieldnames or []
            # Adiciona novas chaves que estão em `data` mas não no CSV
            for key in data.keys():
                if key not in fieldnames:
                    fieldnames.append(key)
    else:
        fieldnames = list(data.keys())

    # Reescreve o CSV inteiro se foi necessário atualizar os cabeçalhos
    if file_exists:
        with open(file_path, "r", encoding="utf-8") as f:
            rows = list(csv.DictReader(f))

        # Atualiza todas as linhas antigas com as novas chaves (valores vazios)
        for row in rows:
            for key in fieldnames:
                if key not in row:
                    row[key] = ""

        # Adiciona a nova linha
        clean_data = {key: data.get(key, "") for key in fieldnames}
        rows.append(clean_data)

        with open(file_path, "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(rows)
    else:
        with open(file_path, "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerow(data)


class DataLoader:
    def __init__(self, base_path):
        self.base_path = Path(base_path)

    def get_categories(self):
        return [self.base_path / path for path in os.listdir(self.base_path)]

    def get_subcategories(self, category_path):
        return [category_path / path for path in os.listdir(category_path)]

    def get_word_paths(self, subcategory_path):
        return [subcategory_path / path for path in os.listdir(subcategory_path)]

    def get_all_words(self, subcategory_name=None):
        palavras = []
        for categoria in self.get_categories():
            for subcat in self.get_subcategories(categoria):
                if subcategory_name and subcat.name != subcategory_name:
                    continue
                for word_path in self.get_word_paths(subcat):
                    palavras.append(self._carregar_palavra(word_path))
        return palavras

    def _carregar_palavra(self, path):
        files = list(os.listdir(path))
        text_info = self._carregar_texto(path)

        return {
            "path": path,
            "files": files,
            "text_pt_br": text_info.get("pergunta_pt-br", ""),
            "text_eng": text_info.get("tradução_en", ""),
            "audio_path": path / "audio.wav",
            "image_figure": path / "image_figure.jpg",
        }

    def _carregar_texto(self, path):
        try:
            with open(path / "text_v2.json", "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception:
            return {}

class GameTimer:
    def __init__(self, master, x=1.0, y=0.0, anchor="ne", font=("Arial", 12), fg="black"):
        self.master = master
        self.label = tk.Label(master, font=font, fg=fg)
        self.label.place(relx=x, rely=y, anchor=anchor)

        self.start_time = None
        self.elapsed_time = 0  
        self.accumulated_time = 0 # Acumula tempo antes de pausas
        self.running = False

        # Botões
        self.play_button = tk.Button(master, text="▶ Play", command=self.play)
        self.play_button.place(relx=x, rely=y + 0.05, anchor=anchor)

        self.stop_button = tk.Button(master, text="⏸ Stop", command=self.stop, state=tk.DISABLED)
        self.stop_button.place(relx=x, rely=y + 0.10, anchor=anchor)

        self.restart_button = tk.Button(master, text="🔄 Reset", command=self.reset_timer)
        self.restart_button.place(relx=x, rely=y + 0.15, anchor=anchor)

    def update_timer(self):
        if self.running:
            current_time = time.time()
            self.elapsed_time = current_time - self.start_time
            total_elapsed = int(self.accumulated_time + self.elapsed_time)

            mins, secs = divmod(total_elapsed, 60)
            time_str = f"Tempo: {mins:02d}:{secs:02d}"

            self.label.config(text=time_str)
            self.master.after(1000, self.update_timer)

    def play(self):
        if not self.running:
            self.running = True
            self.start_time = time.time()
            self.update_timer()
            self.play_button.config(state=tk.DISABLED)
            self.stop_button.config(state=tk.NORMAL)

    def stop(self):
        if self.running:
            self.running = False
            self.accumulated_time += time.time() - self.start_time
            self.play_button.config(state=tk.NORMAL)
            self.stop_button.config(state=tk.DISABLED)

    def reset_timer(self):
        self.running = False
        self.start_time = None
        self.accumulated_time = 0
        self.label.config(text="Tempo: 00:00")
        self.play_button.config(state=tk.NORMAL)
        self.stop_button.config(state=tk.DISABLED)



class HangmanGame(tk.Frame):
    MAX_ATTEMPTS = 6

    def __init__(self, parent=None, **kwargs):
        super().__init__(parent, **kwargs)
        self.parent = parent
        self.loader = DataLoader(base_path=DATA_PATH)
        self.list_data_words:list[dict] = self.loader.get_all_words(subcategory_name=SUBCATEGORY_NAME)
        random.shuffle(self.list_data_words)

        self.timer = GameTimer(self)

        self.clicks_on_guess = 0

        self.id_game = self.gerar_hash_id()
        self.reset_game()
        self.setup_ui()

    def gerar_hash_id(self):
        agora = datetime.now().strftime("%Y%m%d%H%M%S%f")  # AnoMesDiaHoraMinSegMicroseg
        return hex(abs(hash(agora)))[2:]  # Converte para hexadecimal e remove '0x'

    def reset_game(self):
        if not self.list_data_words:
            messagebox.showinfo("Fim", "Não há mais palavras disponíveis.")
            self.parent.quit()
            return

        
        self.dict_info_words = self.list_data_words.pop(random.randint(0, len(self.list_data_words)-1))

        self.word_answer = self.dict_info_words.get("text_eng", None)
        
        self.word_question = self.dict_info_words.get("text_pt_br", None) 

        self.guessed_word = self.hide_text(self.word_answer) 
        self.remaining_attempts = self.MAX_ATTEMPTS
        self.guessed_letters = set()

        
        self.timer.play()

        # self.start_time = time.time()

    def hide_text(self, text: str) -> str:
        # return [" " if c == " " else "_" for c in text]
        return ['_' if c.isalpha() else c for c in text]

    def show_label_words(self):
        font=("Arial", 12)
        fg="black"
        self.palavras_restantes_label = tk.Label(self, text=f"Palavras restantes: {len(self.list_data_words)}", font=font, fg=fg)
        self.palavras_restantes_label.place(relx=0, rely=0.0, anchor="nw")

    def setup_ui(self):

        self.show_label_words()

        self.word_label = tk.Label(self, text=" ".join(self.guessed_word), font=TITLE_FONT)
        self.word_label.pack(pady=10)

        self.attempts_label = tk.Label(self, text="", font=LABEL_FONT)
        self.attempts_label.pack()

        self.guessed_label = tk.Label(self, text="", font=SMALL_FONT)
        self.guessed_label.pack()

        self.image_label = tk.Label(self)
        self.image_label.pack(pady=10)

        self.hint_label = tk.Label(self, text="", font=LABEL_FONT)
        self.hint_label.pack()

        self.input_frame = tk.Frame(self)
        self.input_frame.pack(pady=10)

        tk.Label(self.input_frame, text="Letra:", font=SMALL_FONT).pack(side="left")
        self.letter_entry = tk.Entry(self.input_frame, width=5, font=LABEL_FONT)
        self.letter_entry.pack(side="left")

        self.guess_button = tk.Button(self.input_frame, text="Adivinhar", command=self.check_guess)
        self.guess_button.pack(side="left", padx=5)

        self.update_ui()

    def update_ui(self):
        
        self.palavras_restantes_label.config(text=f"Palavras restantes: {len(self.list_data_words)}")

        self.word_label.config(text=" ".join(self.guessed_word))
        self.attempts_label.config(text=f"Tentativas restantes: {self.remaining_attempts}")
        self.guessed_label.config(text=f"Letras tentadas: {', '.join(sorted(self.guessed_letters))}")
        self.hint_label.config(text=f"Dica: {self.word_question}")

        # if not img_path.exists():
        #     print(f"[ERRO] Caminho da imagem não encontrado: {img_path}")

        # if not hasattr(self.image_label, 'current_image_path') or self.image_label.current_image_path != img_path:

        image_figure = self.dict_info_words.get("image_figure", None)

        if image_figure:
            img_path = image_figure
            img = Image.open(img_path).resize((200, 200))
            photo = ImageTk.PhotoImage(img)

            self.image_label.config(image=photo)
            self.image_label.image = photo  # <- MANTÉM REFERÊNCIA
        else:
            self.image_label.config(image='')
            self.image_label.image = None

    def check_guess(self):

        self.clicks_on_guess += 1

        guess = self.letter_entry.get().strip().lower()
        self.letter_entry.delete(0, tk.END)

        if not guess.isalpha():
            messagebox.showwarning("Letra inválida", "Digite apenas letras.")
            return

        for letter in guess:
            if letter in self.guessed_letters:
                continue

            self.guessed_letters.add(letter)

            if letter in self.word_answer:
                self.reveal_letters(letter)
            else:
                self.remaining_attempts -= 1

            if self.check_game_end():
                return

        self.update_ui()

    def reveal_letters(self, letter):
        for idx, char in enumerate(self.word_answer):
            if char == letter:
                self.guessed_word[idx] = letter

    def check_game_end(self) -> bool:
        if "_" not in self.guessed_word:
            self.save_score(True)
            messagebox.showinfo("Você venceu!", f"A palavra era: {self.word_answer}")
            self.restart_game()
            return True

        if self.remaining_attempts <= 0:
            self.save_score(False)

            # Coloca a palavra de volta na lista
            self.list_data_words.append(self.dict_info_words)
            
            messagebox.showerror("Fim de jogo", f"A palavra era: {self.word_answer}")
            self.restart_game()
            return True

        return False

    def restart_game(self):
        self.reset_game()
        self.update_ui()

    def save_score(self, won: bool):
        attempts_used = self.MAX_ATTEMPTS - self.remaining_attempts
        difficulty = attempts_used / self.MAX_ATTEMPTS

        correct_guessed_letters=[letter for letter in self.guessed_word if letter.isalnum()]
        incorrect_guessed_letters = list(set(self.guessed_letters).difference(set(self.guessed_word)))

        # finaliza_tempo
        # end_time = time.time()
        time_taken = self.timer.elapsed_time

        clicks_on_guess = self.clicks_on_guess

        path_word = self.dict_info_words.get("path", None)
        category = path_word.parts[-3]
        sub_category = path_word.parts[-2]
        # adjetivos/sobre_as_pessoas/unable

        if not any([category == loader_category.name for loader_category in self.loader.get_categories()]):
            print("Categoria Não Existe")

        self.timer.reset_timer()
        self.clicks_on_guess = 0

        # assert total_attempts >= used_attempts, "Tentativas usadas não podem ser maiores que o total permitido."

        datetime_now = datetime.now().isoformat(timespec="seconds")
        

        game_data = {
            "id_game": self.id_game,
            "datetime": datetime_now,
            "word": self.word_answer,
            "category":category,
            "sub_category":sub_category,
            "hint": self.word_question,
            "won": won,
            "difficulty": difficulty,
            "total_attempts": self.MAX_ATTEMPTS,
            "used_attempts": attempts_used,
            "clicks_on_guess": clicks_on_guess,
            "correct_guessed_letters": correct_guessed_letters,
            "incorrect_guessed_letters": incorrect_guessed_letters,
            # "correct_guesses": correct_guesses,
            # "incorrect_guesses": incorrect_guesses,
            "time_taken": time_taken,
            "game_name": "hangman",
        }

        save_game_data(game_data)
        # self.save_score()

    def translate_sentence(self, sentence, source_lang="en", target_lang="pt"):
        try:
            translator = GoogleTranslator(source=source_lang, target=target_lang)
            return translator.translate(sentence)
        except Exception as e:
            print(f"Translation error: {e}")
            return "Translation unavailable"

if __name__ == "__main__":
    try:
        root = tk.Tk()
        root.title("Jogo da Forca com Imagens")
        game = HangmanGame(root)
        game.pack(expand=True, fill="both")
        root.geometry("600x600")
        root.mainloop()
    except Exception as e:
        print(f"Error {e}")
    
    finally:
        root.destroy()


TclError: can't invoke "destroy" command: application has been destroyed

## Futuras features

- taxa de erros
- ultimas palavras [acerto/erro] [audio]
- palavras já vistas [gral de conhecimento]
- completude por categoria, depois subcategoria [conhecidas/desconhecidas]

- melhores formas de esconher as caregorias pra jogar 

- criar paineis de gráfico

In [77]:
import tkinter as tk

def ouvir_audio():
    print("Tocando áudio...")  # Aqui você chamaria a função real de áudio
    popup.destroy()

def mostrar_mensagem():
    global popup
    popup = tk.Toplevel(root)
    popup.title("Mensagem")
    popup.geometry("300x150")
    popup.grab_set()  # Faz a janela se comportar como modal

    label = tk.Label(popup, text="Você errou a palavra!")
    label.pack(pady=10)

    botao_ouvir = tk.Button(popup, text="🔊 Ouvir áudio", command=ouvir_audio)
    botao_ouvir.pack(pady=5)

    botao_ok = tk.Button(popup, text="OK", command=popup.destroy)
    botao_ok.pack(pady=5)

root = tk.Tk()
tk.Button(root, text="Mostrar mensagem", command=mostrar_mensagem).pack(pady=20)
root.mainloop()


In [76]:
import tkinter as tk
from tkinter import messagebox

# Funções dos botões
def marcar_errado():
    messagebox.showinfo("Ação", "Palavra marcada como errada.")

def editar_palavra():
    messagebox.showinfo("Ação", "Abrindo editor de atributos...")

def favoritar_palavra():
    messagebox.showinfo("Ação", "Palavra favoritada!")

# Criação da janela
root = tk.Tk()
root.title("Exemplo de Botões com Emojis")
root.geometry("300x150")

# Frame para os botões
frame = tk.Frame(root)
frame.pack(pady=20)

# Botões com emojis (pode ajustar os emojis se preferir)
btn_errado = tk.Button(frame, text="❌ | Errado", command=marcar_errado)
btn_editar = tk.Button(frame, text="✏️ Editar", command=editar_palavra)
btn_favoritar = tk.Button(frame, text="⭐ Favoritar", command=favoritar_palavra)

# Posicionamento
btn_errado.grid(row=0, column=0, padx=5)
btn_editar.grid(row=0, column=1, padx=5)
btn_favoritar.grid(row=0, column=2, padx=5)

# Loop principal
root.mainloop()


invalid command name "128924614680896update_timer"
    while executing
"128924614680896update_timer"
    ("after" script)


'01/05/2025, 07:19:08'

---

In [ ]:
import tkinter as tk

class App(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title("Exemplo de Mudar de Tela")
        self.geometry("400x300")

        # Tela 1
        self.tela_inicial = tk.Frame(self)
        self.tela_inicial.pack(fill="both", expand=True)

        label = tk.Label(self.tela_inicial, text="Tela Inicial", font=("Arial", 18))
        label.pack(pady=20)

        botao_ir_para_tela2 = tk.Button(self.tela_inicial, text="Ir para Tela 2", command=self.mostrar_tela2)
        botao_ir_para_tela2.pack()

        # Tela 2
        self.tela2 = tk.Frame(self)

        label2 = tk.Label(self.tela2, text="Você está na Tela 2!", font=("Arial", 18))
        label2.pack(pady=20)

        botao_voltar = tk.Button(self.tela2, text="Voltar para Tela Inicial", command=self.mostrar_tela_inicial)
        botao_voltar.pack()

    def mostrar_tela2(self):
        self.tela_inicial.pack_forget()
        self.tela2.pack(fill="both", expand=True)

    def mostrar_tela_inicial(self):
        self.tela2.pack_forget()
        self.tela_inicial.pack(fill="both", expand=True)

if __name__ == "__main__":
    app = App()
    app.mainloop()


In [ ]:
import pandas as pd
import os

In [ ]:
os.chdir("..")

In [ ]:
df = pd.read_csv("database/csv/words_oxford_levels.csv")

WORDS = df.sample(100)["Word"].tolist()

FileNotFoundError: [Errno 2] No such file or directory: 'database/csv/words_oxford_levels.csv'

In [ ]:
from deep_translator import GoogleTranslator



WORDS = ['Red', 'Blue', 'Green', 'Pink', 'Black',
                        'Yellow', 'Orange', 'White', 'Purple', 'Brown']